DOWNLOAD DATASETS from:
STOCK PRICES:
https://www.kaggle.com/ehallmar/daily-historical-stock-prices-1970-2018 
VIX VALUES:
https://datahub.io/core/finance-vix


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('./data/historical_stock_prices.csv')

In [3]:
df.head()

,ticker,open,close,adj_close,low,high,volume,date
0,AHH,11.50,11.58,8.493155,11.25,11.68,4633900,2013-05-08
1,AHH,11.66,11.55,8.471151,11.50,11.66,275800,2013-05-09
2,AHH,11.55,11.60,8.507822,11.50,11.60,277100,2013-05-10
3,AHH,11.63,11.65,8.544494,11.55,11.65,147400,2013-05-13
4,AHH,11.60,11.53,8.456484,11.50,11.60,184100,2013-05-14


In [4]:
df.shape

(20973889, 8)

In [83]:
df.isnull().sum()

ticker       0
open         0
close        0
adj_close    0
low          0
high         0
volume       0
date         0
dtype: int64

We import the dataset of stock data since the 1980s. There are over 20,000,000 datapoints which will be far too much for us to predict on, and would lead to highly imbalanced classes. As such, we decided to group the top 3 stocks from each sector of the stock market (which totals 11 sectors) into individual dataframes.

UTILITIES

In [5]:
dffe = df.loc[df['ticker'] == 'FE']
dfaes = df.loc[df['ticker'] == 'AES']
dfnee = df.loc[df['ticker'] == 'NEE']

We pull out each individual stock into there own dataframe according to ticker.

In [6]:
dfutilities = dffe.merge(dfaes, on='date')
dfutilities = dfutilities.merge(dfnee, on='date')

We merge the three dataframes into a single dataframe: dfutilities, on the date column.

In [7]:
dfutilities.head()

,ticker_x,open_x,close_x,adj_close_x,low_x,high_x,volume_x,date,ticker_y,open_y,...,low_y,high_y,volume_y,ticker,open,close,adj_close,low,high,volume
0,FE,25.7500,25.1875,9.657098,25.1250,25.7500,947200,1997-11-10,AES,20.25000,...,20.25000,20.59375,341200,NEE,26.18750,26.53125,9.480980,26.18750,26.53125,939400
1,FE,25.1875,25.3125,9.705027,25.1250,25.4375,514700,1997-11-11,AES,20.62500,...,20.31250,20.81250,522800,NEE,26.59375,27.03125,9.659657,26.56250,27.03125,1238400
2,FE,25.2500,25.6250,9.824841,25.1250,25.8750,344700,1997-11-12,AES,20.59375,...,20.50000,20.75000,644800,NEE,26.90625,27.21875,9.726656,26.87500,27.43750,1367600
3,FE,25.5625,25.8750,9.920691,25.5625,25.9375,1007700,1997-11-13,AES,20.59375,...,20.15625,20.62500,761600,NEE,27.21875,27.06250,9.670826,27.00000,27.37500,972200
4,FE,25.8750,26.0000,9.968615,25.6875,26.0000,1394900,1997-11-14,AES,20.12500,...,18.59375,20.12500,894000,NEE,26.93750,26.96875,9.637321,26.78125,27.15625,1103800


In [8]:
dfutilities['open utilities'] = dfutilities['open_x'] + dfutilities['open_y'] + dfutilities['open']

dfutilities.drop(columns = ['open_x', 'open_y', 'open'], inplace = True)

dfutilities['close utilities'] = dfutilities['close_x'] + dfutilities['close_y'] + dfutilities['close']

dfutilities.drop(columns = ['close_x', 'close_y', 'close'], inplace = True)

dfutilities['adj close utilities'] = dfutilities['adj_close_x'] + dfutilities['adj_close_y'] + dfutilities['adj_close']

dfutilities.drop(columns = ['adj_close_x', 'adj_close_y', 'adj_close'], inplace = True)

dfutilities['low utilities'] = dfutilities['low_x'] + dfutilities['low_y'] + dfutilities['low']

dfutilities.drop(columns = ['low_x', 'low_y', 'low'], inplace = True)

dfutilities['high utilities'] = dfutilities['high_x'] + dfutilities['high_y'] + dfutilities['high']

dfutilities.drop(columns = ['high_x', 'high_y', 'high'], inplace = True)

dfutilities['volume utilities'] = dfutilities['volume_x'] + dfutilities['volume_y'] + dfutilities['volume']

dfutilities.drop(columns = ['volume_x', 'volume_y', 'volume'], inplace = True)

dfutilities.drop(columns = ['ticker_x', 'ticker_y', 'ticker'], inplace = True)

We add the shared features of each stock together, so for example open_x, open_y, and open we added each value together into open utilities. This groups all the features together by sector. We then drop open_x, open_y, and open, so the dataframe only has the summed values of the sector.

This will help us identify features later when we combine all the different sector dataframes together into a single dataframe. 

In [9]:
dfutilities.head()

,date,open utilities,close utilities,adj close utilities,low utilities,high utilities,volume utilities
0,1997-11-10,72.18750,72.21875,36.100451,71.56250,72.87500,2227800
1,1997-11-11,72.40625,73.00000,36.456347,72.00000,73.28125,2275900
2,1997-11-12,72.75000,73.40625,36.565586,72.50000,74.06250,2357100
3,1997-11-13,73.37500,73.09375,36.269456,72.71875,73.93750,2741500
4,1997-11-14,72.93750,71.78125,35.172016,71.06250,73.28125,3392700


We then do this process for the top 3 stocks of each of the 11 sectors.

HEALTH CARE

In [11]:
dffe = df.loc[df['ticker'] == 'PFE']
dfaes = df.loc[df['ticker'] == 'BMY']
dfnee = df.loc[df['ticker'] == 'MRK']

In [12]:
dfhealth = dffe.merge(dfaes, on='date')
dfhealth = dfhealth.merge(dfnee, on='date')

In [13]:
dfhealth['open health'] = dfhealth['open_x'] + dfhealth['open_y'] + dfhealth['open']
dfhealth.drop(columns = ['open_x', 'open_y', 'open'], inplace = True)
dfhealth['close health'] = dfhealth['close_x'] + dfhealth['close_y'] + dfhealth['close']
dfhealth.drop(columns = ['close_x', 'close_y', 'close'], inplace = True)
dfhealth['adj close health'] = dfhealth['adj_close_x'] + dfhealth['adj_close_y'] + dfhealth['adj_close']
dfhealth.drop(columns = ['adj_close_x', 'adj_close_y', 'adj_close'], inplace = True)
dfhealth['low health'] = dfhealth['low_x'] + dfhealth['low_y'] + dfhealth['low']
dfhealth.drop(columns = ['low_x', 'low_y', 'low'], inplace = True)
dfhealth['high health'] = dfhealth['high_x'] + dfhealth['high_y'] + dfhealth['high']
dfhealth.drop(columns = ['high_x', 'high_y', 'high'], inplace = True)
dfhealth['volume health'] = dfhealth['volume_x'] + dfhealth['volume_y'] + dfhealth['volume']
dfhealth.drop(columns = ['volume_x', 'volume_y', 'volume'], inplace = True)
dfhealth.drop(columns = ['ticker_x', 'ticker_y', 'ticker'], inplace = True)

In [14]:
dfhealth.head()

,date,open health,close health,adj close health,low health,high health,volume health
0,1972-06-01,4.743488,4.760473,0.439316,4.722655,4.775098,5864900
1,1972-06-02,4.753528,4.733923,0.433715,4.705785,4.782157,3128400
2,1972-06-05,4.733923,4.724620,0.431966,4.694615,4.747681,3930700
3,1972-06-06,4.724620,4.707390,0.427157,4.672668,4.769628,4026100
4,1972-06-07,4.707390,4.707144,0.427646,4.665854,4.735659,2496300


FINANCIALS

In [16]:
dffe = df.loc[df['ticker'] == 'WFC']
dfaes = df.loc[df['ticker'] == 'BAC']
dfnee = df.loc[df['ticker'] == 'JPM']

In [17]:
dffinance = dffe.merge(dfaes, on='date')
dffinance = dffinance.merge(dfnee, on='date')

In [18]:
dffinance['open finance'] = dffinance['open_x'] + dffinance['open_y'] + dffinance['open']
dffinance.drop(columns = ['open_x', 'open_y', 'open'], inplace = True)
dffinance['close finance'] = dffinance['close_x'] + dffinance['close_y'] + dffinance['close']
dffinance.drop(columns = ['close_x', 'close_y', 'close'], inplace = True)
dffinance['adj close finance'] = dffinance['adj_close_x'] + dffinance['adj_close_y'] + dffinance['adj_close']
dffinance.drop(columns = ['adj_close_x', 'adj_close_y', 'adj_close'], inplace = True)
dffinance['low finance'] = dffinance['low_x'] + dffinance['low_y'] + dffinance['low']
dffinance.drop(columns = ['low_x', 'low_y', 'low'], inplace = True)
dffinance['high finance'] = dffinance['high_x'] + dffinance['high_y'] + dffinance['high']
dffinance.drop(columns = ['high_x', 'high_y', 'high'], inplace = True)
dffinance['volume finance'] = dffinance['volume_x'] + dffinance['volume_y'] + dffinance['volume']
dffinance.drop(columns = ['volume_x', 'volume_y', 'volume'], inplace = True)
dffinance.drop(columns = ['ticker_x', 'ticker_y', 'ticker'], inplace = True)

In [19]:
dffinance.head()

,date,open finance,close finance,adj close finance,low finance,high finance,volume finance
0,1980-03-17,7.284143,7.255787,0.422293,7.216435,7.457755,1912000
1,1980-03-18,7.255787,7.303241,0.423389,7.224537,7.355903,443900
2,1980-03-19,7.355903,7.403356,0.431583,7.329861,7.458333,2395700
3,1980-03-20,7.403356,7.361111,0.429308,7.342593,7.413773,462900
4,1980-03-21,7.361111,7.472222,0.436077,7.345486,7.493056,909200


INFORMATION TECHNOLOGY

In [21]:
dffe = df.loc[df['ticker'] == 'AAPL']
dfaes = df.loc[df['ticker'] == 'AMD']
dfnee = df.loc[df['ticker'] == 'MU']

In [22]:
dfIT = dffe.merge(dfaes, on='date')
dfIT = dfIT.merge(dfnee, on='date')

In [23]:
dfIT['open IT'] = dfIT['open_x'] + dfIT['open_y'] + dfIT['open']
dfIT.drop(columns = ['open_x', 'open_y', 'open'], inplace = True)
dfIT['close IT'] = dfIT['close_x'] + dfIT['close_y'] + dfIT['close']
dfIT.drop(columns = ['close_x', 'close_y', 'close'], inplace = True)
dfIT['adj close IT'] = dfIT['adj_close_x'] + dfIT['adj_close_y'] + dfIT['adj_close']
dfIT.drop(columns = ['adj_close_x', 'adj_close_y', 'adj_close'], inplace = True)
dfIT['low IT'] = dfIT['low_x'] + dfIT['low_y'] + dfIT['low']
dfIT.drop(columns = ['low_x', 'low_y', 'low'], inplace = True)
dfIT['high IT'] = dfIT['high_x'] + dfIT['high_y'] + dfIT['high']
dfIT.drop(columns = ['high_x', 'high_y', 'high'], inplace = True)
dfIT['volume IT'] = dfIT['volume_x'] + dfIT['volume_y'] + dfIT['volume']
dfIT.drop(columns = ['volume_x', 'volume_y', 'volume'], inplace = True)
dfIT.drop(columns = ['ticker_x', 'ticker_y', 'ticker'], inplace = True)

In [24]:
dfIT.head()

,date,open IT,close IT,adj close IT,low IT,high IT,volume IT
0,1984-06-01,17.937054,18.142411,17.475508,17.872321,18.217411,68669400
1,1984-06-04,18.329911,17.879018,17.224903,17.812054,18.474107,41899800
2,1984-06-05,17.807589,17.897768,17.274810,17.583036,18.120089,84986400
3,1984-06-06,17.972768,18.492857,17.842806,17.845536,18.520089,42703800
4,1984-06-07,18.467857,18.713393,18.050465,18.264732,18.795089,28251400


TELECOMMUNICATION SERVICES 

In [26]:
dffe = df.loc[df['ticker'] == 'T']
dfaes = df.loc[df['ticker'] == 'VZ']
dfnee = df.loc[df['ticker'] == 'DIS']

In [27]:
dffe.shape, dfaes.shape, dfnee.shape

((8764, 8), (8764, 8), (12274, 8))

In [28]:
dftelecom = dffe.merge(dfaes, on='date')
dftelecom = dftelecom.merge(dfnee, on='date')

In [29]:
dftelecom['open telecom'] = dftelecom['open_x'] + dftelecom['open_y'] + dftelecom['open']
dftelecom.drop(columns = ['open_x', 'open_y', 'open'], inplace = True)
dftelecom['close telecom'] = dftelecom['close_x'] + dftelecom['close_y'] + dftelecom['close']
dftelecom.drop(columns = ['close_x', 'close_y', 'close'], inplace = True)
dftelecom['adj close telecom'] = dftelecom['adj_close_x'] + dftelecom['adj_close_y'] + dftelecom['adj_close']
dftelecom.drop(columns = ['adj_close_x', 'adj_close_y', 'adj_close'], inplace = True)
dftelecom['low telecom'] = dftelecom['low_x'] + dftelecom['low_y'] + dftelecom['low']
dftelecom.drop(columns = ['low_x', 'low_y', 'low'], inplace = True)
dftelecom['high telecom'] = dftelecom['high_x'] + dftelecom['high_y'] + dftelecom['high']
dftelecom.drop(columns = ['high_x', 'high_y', 'high'], inplace = True)
dftelecom['volume telecom'] = dftelecom['volume_x'] + dftelecom['volume_y'] + dftelecom['volume']
dftelecom.drop(columns = ['volume_x', 'volume_y', 'volume'], inplace = True)
dftelecom.drop(columns = ['ticker_x', 'ticker_y', 'ticker'], inplace = True)

In [30]:
dftelecom.head()

,date,open telecom,close telecom,adj close telecom,low telecom,high telecom,volume telecom
0,1983-11-21,14.026838,14.047038,0.990941,13.464761,14.152618,19687900
1,1983-11-22,14.047038,13.969563,0.984588,13.945239,14.167158,15077100
2,1983-11-23,13.966995,13.898869,0.978623,13.823039,14.005515,16651900
3,1983-11-25,13.898869,13.817270,0.972382,13.799148,13.944171,12745000
4,1983-11-28,13.775114,13.546703,0.958125,13.513463,13.798372,17736400


REAL ESTATE

In [32]:
dffe = df.loc[df['ticker'] == 'HST']
dfaes = df.loc[df['ticker'] == 'O']
dfnee = df.loc[df['ticker'] == 'WY']

In [33]:
dfrealestate = dffe.merge(dfaes, on='date')
dfrealestate = dfrealestate.merge(dfnee, on='date')

In [34]:
dfrealestate['open realestate'] = dfrealestate['open_x'] + dfrealestate['open_y'] + dfrealestate['open']
dfrealestate.drop(columns = ['open_x', 'open_y', 'open'], inplace = True)
dfrealestate['close realestate'] = dfrealestate['close_x'] + dfrealestate['close_y'] + dfrealestate['close']
dfrealestate.drop(columns = ['close_x', 'close_y', 'close'], inplace = True)
dfrealestate['adj close realestate'] = dfrealestate['adj_close_x'] + dfrealestate['adj_close_y'] + dfrealestate['adj_close']
dfrealestate.drop(columns = ['adj_close_x', 'adj_close_y', 'adj_close'], inplace = True)
dfrealestate['low realestate'] = dfrealestate['low_x'] + dfrealestate['low_y'] + dfrealestate['low']
dfrealestate.drop(columns = ['low_x', 'low_y', 'low'], inplace = True)
dfrealestate['high realestate'] = dfrealestate['high_x'] + dfrealestate['high_y'] + dfrealestate['high']
dfrealestate.drop(columns = ['high_x', 'high_y', 'high'], inplace = True)
dfrealestate['volume realestate'] = dfrealestate['volume_x'] + dfrealestate['volume_y'] + dfrealestate['volume']
dfrealestate.drop(columns = ['volume_x', 'volume_y', 'volume'], inplace = True)
dfrealestate.drop(columns = ['ticker_x', 'ticker_y', 'ticker'], inplace = True)

In [35]:
dfrealestate.head()

,date,open realestate,close realestate,adj close realestate,low realestate,high realestate,volume realestate
0,1994-10-18,58.392621,58.001652,24.051823,57.408590,58.564152,1515700
1,1994-10-19,57.721090,58.939152,24.229704,57.112060,59.126652,1833200
2,1994-10-20,58.846090,58.642621,24.069210,58.065529,59.205121,1118100
3,1994-10-21,58.299560,58.517621,24.058503,57.768998,59.080121,1117100
4,1994-10-24,57.924560,56.689152,23.393955,56.253029,58.564152,2045900


CONSUMER STAPLES

In [37]:
dffe = df.loc[df['ticker'] == 'WMT']
dfaes = df.loc[df['ticker'] == 'MDLZ']
dfnee = df.loc[df['ticker'] == 'KO']

In [38]:
dfconsumerstaples = dffe.merge(dfaes, on='date')
dfconsumerstaples = dfconsumerstaples.merge(dfnee, on='date')

In [39]:
dfconsumerstaples['open consumerstaples'] = dfconsumerstaples['open_x'] + dfconsumerstaples['open_y'] + dfconsumerstaples['open']
dfconsumerstaples.drop(columns = ['open_x', 'open_y', 'open'], inplace = True)
dfconsumerstaples['close consumerstaples'] = dfconsumerstaples['close_x'] + dfconsumerstaples['close_y'] + dfconsumerstaples['close']
dfconsumerstaples.drop(columns = ['close_x', 'close_y', 'close'], inplace = True)
dfconsumerstaples['adj close consumerstaples'] = dfconsumerstaples['adj_close_x'] + dfconsumerstaples['adj_close_y'] + dfconsumerstaples['adj_close']
dfconsumerstaples.drop(columns = ['adj_close_x', 'adj_close_y', 'adj_close'], inplace = True)
dfconsumerstaples['low consumerstaples'] = dfconsumerstaples['low_x'] + dfconsumerstaples['low_y'] + dfconsumerstaples['low']
dfconsumerstaples.drop(columns = ['low_x', 'low_y', 'low'], inplace = True)
dfconsumerstaples['high consumerstaples'] = dfconsumerstaples['high_x'] + dfconsumerstaples['high_y'] + dfconsumerstaples['high']
dfconsumerstaples.drop(columns = ['high_x', 'high_y', 'high'], inplace = True)
dfconsumerstaples['volume consumerstaples'] = dfconsumerstaples['volume_x'] + dfconsumerstaples['volume_y'] + dfconsumerstaples['volume']
dfconsumerstaples.drop(columns = ['volume_x', 'volume_y', 'volume'], inplace = True)
dfconsumerstaples.drop(columns = ['ticker_x', 'ticker_y', 'ticker'], inplace = True)

In [40]:
dfconsumerstaples.head()

,date,open consumerstaples,close consumerstaples,adj close consumerstaples,low consumerstaples,high consumerstaples,volume consumerstaples
0,2001-06-13,105.049999,104.325001,60.168246,103.850002,105.549999,78666300
1,2001-06-14,103.745001,102.160000,59.033271,101.320002,104.205000,43556000
2,2001-06-15,101.910002,100.880001,58.081893,100.005001,103.074999,42513500
3,2001-06-18,100.989998,100.725000,58.111563,100.144999,102.415001,17497100
4,2001-06-19,101.955002,101.135000,58.385124,100.610001,102.245001,16308100


ENERGY

In [42]:
df_XOM =  df.loc[df['ticker'] == 'XOM']
df_MRO =  df.loc[df['ticker'] == 'MRO']
df_OXY =  df.loc[df['ticker'] == 'OXY']

df_energy = df_XOM.merge(df_MRO, on='date')

df_energy = df_energy.merge(df_OXY, on = "date")

In [43]:
df_energy["open energy"] = df_energy["open_x"] + df_energy["open_y"] + df_energy["open"]

df_energy["close energy"] = df_energy["close_x"] + df_energy["close_y"] + df_energy["close"]

df_energy["adjusted close energy"] = df_energy["adj_close_x"] + df_energy["adj_close_y"] + df_energy["adj_close"]

df_energy["low energy"] = df_energy["low_x"] + df_energy["low_y"] + df_energy["low"]

df_energy["high energy"] = df_energy["high_x"] + df_energy["high_y"] + df_energy["high"]

df_energy["volume energy"] = df_energy["volume_x"] + df_energy["volume_y"] + df_energy["volume"]

In [44]:
df_energy.drop(columns = ["ticker_x",
"open_x",
"close_x",
"adj_close_x",
"low_x",
"high_x",
"volume_x",
"ticker_y",
"open_y",
"close_y",
"adj_close_y",
"low_y",
"high_y",
"volume_y",
"ticker",
"open",
"close",
"adj_close",
"low",
"high",
"volume"],
inplace = True)

In [45]:
df_energy.head()

,date,open energy,close energy,adjusted close energy,low energy,high energy,volume energy
0,1981-12-31,22.693915,22.828170,1.158364,22.435390,22.905930,4152500
1,1982-01-04,22.827988,22.812363,1.159162,22.614844,22.950183,5617400
2,1982-01-05,22.630287,22.508090,1.144699,22.430330,22.783551,7007000
3,1982-01-06,22.337127,22.412631,1.136917,22.152794,22.505834,10681600
4,1982-01-07,22.212856,22.288360,1.128435,21.982013,22.412631,5334000


MATERIALS

In [47]:
df_FCX =  df.loc[df['ticker'] == 'FCX']
df_IFF =  df.loc[df['ticker'] == 'IFF']
df_NEM =  df.loc[df['ticker'] == 'NEM']

df_materials = df_FCX.merge(df_IFF, on='date')

df_materials = df_materials.merge(df_NEM, on = "date")

In [48]:
df_materials["open materials"] = df_materials["open_x"] + df_materials["open_y"] + df_materials["open"]

df_materials["close materials"] = df_materials["close_x"] + df_materials["close_y"] + df_materials["close"]

df_materials["adjusted close materials"] = df_materials["adj_close_x"] + df_materials["adj_close_y"] + df_materials["adj_close"]

df_materials["low materials"] = df_materials["low_x"] + df_materials["low_y"] + df_materials["low"]

df_materials["high materials"] = df_materials["high_x"] + df_materials["high_y"] + df_materials["high"]

df_materials["volume materials"] = df_materials["volume_x"] + df_materials["volume_y"] + df_materials["volume"]

In [49]:
df_materials.drop(columns = ["ticker_x",
"open_x",
"close_x",
"adj_close_x",
"low_x",
"high_x",
"volume_x",
"ticker_y",
"open_y",
"close_y",
"adj_close_y",
"low_y",
"high_y",
"volume_y",
"ticker",
"open",
"close",
"adj_close",
"low",
"high",
"volume"],
inplace = True)

INDUSTRIALS

In [51]:
df_GE =  df.loc[df['ticker'] == 'GE']
df_AAL =  df.loc[df['ticker'] == 'AAL']
df_UAL =  df.loc[df['ticker'] == 'UAL']

df_industrials = df_GE.merge(df_AAL, on='date')

df_industrials = df_industrials.merge(df_UAL, on = "date")

In [52]:
df_industrials["open industrials"] = df_industrials["open_x"] + df_industrials["open_y"] + df_industrials["open"]

df_industrials["close industrials"] = df_industrials["close_x"] + df_industrials["close_y"] + df_industrials["close"]

df_industrials["adjusted close industrials"] = df_industrials["adj_close_x"] + df_industrials["adj_close_y"] + df_industrials["adj_close"]

df_industrials["low industrials"] = df_industrials["low_x"] + df_industrials["low_y"] + df_industrials["low"]

df_industrials["high industrials"] = df_industrials["high_x"] + df_industrials["high_y"] + df_industrials["high"]

df_industrials["volume industrials"] = df_industrials["volume_x"] + df_industrials["volume_y"] + df_industrials["volume"]

In [53]:
df_industrials.drop(columns = ["ticker_x",
"open_x",
"close_x",
"adj_close_x",
"low_x",
"high_x",
"volume_x",
"ticker_y",
"open_y",
"close_y",
"adj_close_y",
"low_y",
"high_y",
"volume_y",
"ticker",
"open",
"close",
"adj_close",
"low",
"high",
"volume"],
inplace = True)

CONSUMER DISCRETIONARY

In [55]:
df_F =  df.loc[df['ticker'] == 'F']
df_CCL =  df.loc[df['ticker'] == 'CCL']
df_TJX =  df.loc[df['ticker'] == 'TJX']

df_consumer_discretionary = df_F.merge(df_CCL, on='date')

df_consumer_discretionary = df_consumer_discretionary.merge(df_TJX, on = "date")

In [56]:
df_consumer_discretionary["open consumer discretionary"] = df_consumer_discretionary["open_x"] + df_consumer_discretionary["open_y"] + df_consumer_discretionary["open"]

df_consumer_discretionary["close consumer discretionary"] = df_consumer_discretionary["close_x"] + df_consumer_discretionary["close_y"] + df_consumer_discretionary["close"]

df_consumer_discretionary["adjusted close consumer discretionary"] = df_consumer_discretionary["adj_close_x"] + df_consumer_discretionary["adj_close_y"] + df_consumer_discretionary["adj_close"]

df_consumer_discretionary["low consumer discretionary"] = df_consumer_discretionary["low_x"] + df_consumer_discretionary["low_y"] + df_consumer_discretionary["low"]

df_consumer_discretionary["high consumer discretionary"] = df_consumer_discretionary["high_x"] + df_consumer_discretionary["high_y"] + df_consumer_discretionary["high"]

df_consumer_discretionary["volume consumer discretionary"] = df_consumer_discretionary["volume_x"] + df_consumer_discretionary["volume_y"] + df_consumer_discretionary["volume"]

In [57]:
df_consumer_discretionary.drop(columns = ["ticker_x",
"open_x",
"close_x",
"adj_close_x",
"low_x",
"high_x",
"volume_x",
"ticker_y",
"open_y",
"close_y",
"adj_close_y",
"low_y",
"high_y",
"volume_y",
"ticker",
"open",
"close",
"adj_close",
"low",
"high",
"volume"],
inplace = True)

In [58]:
df_consumer_discretionary.to_csv('df_consumer_discretionary.csv') 

In [59]:
print(df_consumer_discretionary.shape)
print(df_industrials.shape)
print(df_materials.shape)
print(df_energy.shape)
print(dfconsumerstaples.shape)
print(dfrealestate.shape)
print(dftelecom.shape)
print(dfIT.shape)
print(dffinance.shape)
print(dfhealth.shape)
print(dfutilities.shape)

(7837, 7)
(3161, 7)
(5825, 7)
(9243, 7)
(4328, 7)
(6007, 7)
(8764, 7)
(8630, 7)
(9693, 7)
(11661, 7)
(5232, 7)


In [60]:
dfallsectors = df_consumer_discretionary.merge(df_industrials, on='date')
dfallsectors = dfallsectors.merge(df_materials, on='date')
dfallsectors = dfallsectors.merge(df_energy, on='date')
dfallsectors = dfallsectors.merge(dfconsumerstaples, on='date')
dfallsectors = dfallsectors.merge(dfrealestate, on='date')
dfallsectors = dfallsectors.merge(dftelecom, on='date')
dfallsectors = dfallsectors.merge(dfIT, on='date')
dfallsectors = dfallsectors.merge(dffinance, on='date')
dfallsectors = dfallsectors.merge(dfhealth, on='date')
dfallsectors = dfallsectors.merge(dfutilities, on='date')

In [61]:
dfallsectors.head()

,date,open consumer discretionary,close consumer discretionary,adjusted close consumer discretionary,low consumer discretionary,high consumer discretionary,volume consumer discretionary,open industrials,close industrials,adjusted close industrials,...,adj close health,low health,high health,volume health,open utilities,close utilities,adj close utilities,low utilities,high utilities,volume utilities
0,2006-02-06,72.775002,72.399999,52.607455,71.834998,73.345000,20881200,96.869999,95.920002,80.550484,...,49.798768,80.990002,82.180000,38586900,106.390003,107.010000,68.658760,105.920002,107.280003,3943300
1,2006-02-07,72.414999,71.985001,52.345269,71.624998,72.880000,21343500,95.910002,94.300003,79.161594,...,49.820878,80.870003,82.800003,43082300,106.880001,106.709999,68.506651,105.889997,107.720001,8455900
2,2006-02-08,72.325000,72.639999,52.817276,71.894999,72.744998,29281000,92.870001,94.730001,79.459028,...,51.223830,81.590000,83.670000,78337500,107.520000,107.610001,69.082878,106.410000,107.750000,7349000
3,2006-02-09,72.954999,72.855001,52.963918,72.590001,73.450001,23437700,92.529999,97.769999,82.256662,...,51.143907,83.070000,84.350000,61646500,107.670000,108.390001,69.535818,107.189999,108.800001,6477300
4,2006-02-10,73.025000,72.690001,52.839699,72.159999,73.485002,21519000,97.339996,97.520000,81.905180,...,50.612302,81.230000,83.040001,72039700,108.260000,108.160002,69.375387,107.090002,109.110001,6055000


We combine all the different sectors dataframes into a single concatenated dataframe which we will model on. Next we will merge the data that we are trying to predict: the VIX.

In [62]:
dfvix = pd.read_csv('./data/vix-daily_csv.csv')

In [63]:
dfvix.head()

,Date,VIX Open,VIX High,VIX Low,VIX Close
0,2004-01-02,17.96,18.68,17.54,18.22
1,2004-01-05,18.45,18.49,17.44,17.49
2,2004-01-06,17.66,17.67,16.19,16.73
3,2004-01-07,16.72,16.75,15.50,15.50
4,2004-01-08,15.42,15.68,15.32,15.61


In [64]:
dfvix.drop(columns = ['VIX Open', 'VIX High', 'VIX Low'], inplace = True)

In [65]:
dfvix.rename(columns = {'Date':'date'}, inplace = True) 

In [66]:
dfvix.head()

,date,VIX Close
0,2004-01-02,18.22
1,2004-01-05,17.49
2,2004-01-06,16.73
3,2004-01-07,15.50
4,2004-01-08,15.61


We import the vix data and drop all columns except Date and VIX close. We also rename the Date column to be date, to make it consistent with our dfallsectors naming conventions.

In [67]:
dffinal = dfallsectors.merge(dfvix, on='date')

In [68]:
dffinal.head()

,date,open consumer discretionary,close consumer discretionary,adjusted close consumer discretionary,low consumer discretionary,high consumer discretionary,volume consumer discretionary,open industrials,close industrials,adjusted close industrials,...,low health,high health,volume health,open utilities,close utilities,adj close utilities,low utilities,high utilities,volume utilities,VIX Close
0,2006-02-06,72.775002,72.399999,52.607455,71.834998,73.345000,20881200,96.869999,95.920002,80.550484,...,80.990002,82.180000,38586900,106.390003,107.010000,68.658760,105.920002,107.280003,3943300,13.04
1,2006-02-07,72.414999,71.985001,52.345269,71.624998,72.880000,21343500,95.910002,94.300003,79.161594,...,80.870003,82.800003,43082300,106.880001,106.709999,68.506651,105.889997,107.720001,8455900,13.59
2,2006-02-08,72.325000,72.639999,52.817276,71.894999,72.744998,29281000,92.870001,94.730001,79.459028,...,81.590000,83.670000,78337500,107.520000,107.610001,69.082878,106.410000,107.750000,7349000,12.83
3,2006-02-09,72.954999,72.855001,52.963918,72.590001,73.450001,23437700,92.529999,97.769999,82.256662,...,83.070000,84.350000,61646500,107.670000,108.390001,69.535818,107.189999,108.800001,6477300,13.12
4,2006-02-10,73.025000,72.690001,52.839699,72.159999,73.485002,21519000,97.339996,97.520000,81.905180,...,81.230000,83.040001,72039700,108.260000,108.160002,69.375387,107.090002,109.110001,6055000,12.87


In [69]:
dffinal.shape

(3160, 68)

In order for the number of features to be less than the square root of the number of rows, we will drop columns adj_close. We chose adj_close because, the normal close feature accounts for most of the data in adjusted close. This will allow our model to have a better fit.

In [73]:
dffinal.drop(columns = ['adj close utilities', 'adj close health',
                       'adj close finance', 'adj close IT', 
                       'adj close telecom', 'adj close realestate',
                       'adj close consumerstaples', 'adjusted close energy',
                       'adjusted close materials', 'adjusted close industrials',
                       'adjusted close consumer discretionary'
                      ], inplace = True)

In [74]:
dffinal.shape

(3160, 57)

Next we set the date to a datetime data type, and set it as the index.

In [76]:
dffinal['date'] = pd.to_datetime(dffinal['date'])
dffinal.set_index('date', inplace=True)
dffinal.sort_index(inplace=True)

Finally we save the final dataframe to csv to be imported in the modeling notebook.

In [78]:
dffinal.to_csv('dffinal.csv')